# SQL III - View
## 1. Introduction
### What is a View?
A view is a reusable virtual table that is created by querying existing tables. The virtual table is not saved in memory. Instead, the query used for the view is saved with the name. Each time the view is used, a new query of that view is being executed. 

---

### Why Are Views Important?
- **Privacy**: views can limit the information shared by taking a subset of information from full tables
- **Reusability**: can make repetitive queries easier to write
- **Generalization**: can combine different sub categories into a larger category

---

### Normal Views vs Materialized Views
While normal views do not store the result of a query, materialized views store the results of a query into memory. This has the benefit of increasing the speed of data retrieval but at the cost of memory space and overhead from maintaining data consistency. 

### Setup

In [8]:
# Imports
from tabulate import tabulate
import psycopg2
import time

# Database connection parameters
dbname = 'banking_db'
user = 'postgres'
password = 'postgres'
host = 'postgres_db'  # This should be the service name defined in docker-compose.yml
port = '5432'  

# Establish connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Create a cursor object to interact with the database
cur = conn.cursor()

## 2. Creating Views
#### We will be working with 3 tables, **agents**, **customer**, and **contractors**.


### 2.1 Privacy 
#### I want to give access to all employees in my company the name, working area, and phone number of all agents. 
#### However I don't want to share the agent code and commission. We can achieve this by creating a view. 

In [11]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW public_agents AS 
        SELECT agent_name, working_area, phone_no
        FROM agents 
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [12]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM public_agents")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+--------------+----------------+--------------+
| agent_name   | working_area   | phone_no     |
+==============+================+==============+
| Ramasundar   | Bangalore      | 077-25814763 |
+--------------+----------------+--------------+
| Alex         | London         | 075-12458969 |
+--------------+----------------+--------------+
| Alford       | New York       | 044-25874365 |
+--------------+----------------+--------------+
| Ravi Kumar   | Bangalore      | 077-45625874 |
+--------------+----------------+--------------+
| Santakumar   | Chennai        | 007-22388644 |
+--------------+----------------+--------------+
| Lucida       | San Jose       | 044-52981425 |
+--------------+----------------+--------------+
| Anderson     | Brisbane       | 045-21447739 |
+--------------+----------------+--------------+
| Subbarao     | Bangalore      | 077-12346674 |
+--------------+----------------+--------------+
| Mukesh       | Mumbai         | 029-12358964 |
+--------------+----

#### Employees can now be given access to the view **public_employees** without sharing sensitive information. 
--- 
### 2.2 Reusability 
#### I regularly have to make queries on the customers living in the US and India. To do this, I make a custom view that I can reuse for further queries. 

In [13]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Resets the view
# cur.execute("DROP VIEW customers_USA_india")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW customers_USA_india AS 
        SELECT cust_code, cust_name, cust_city, cust_country, grade, outstanding_amt, agent_code
        FROM customer 
        WHERE CUST_COUNTRY = 'USA' OR CUST_COUNTRY = 'India'
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [14]:
# Query to fetch records
cur.execute("SELECT * FROM customers_USA_india")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| cust_code   | cust_name   | cust_city   | cust_country   |   grade |   outstanding_amt | agent_code   |
+=============+=============+=============+================+=========+===================+==============+
| C00001      | Micheal     | New York    | USA            |       2 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00020      | Albert      | New York    | USA            |       3 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00025      | Ravindran   | Bangalore   | India          |       2 |              8000 | A011         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00002      | Bolt        | New York    | US

#### Now that I have created my view, I can query it just like any table. For example, let's say that I want to find customers with a grade greater or equal to 2. 

In [15]:
# Query to fetch records
cur.execute("""
    SELECT * 
    FROM customers_USA_india
    WHERE grade >= 2
""")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| cust_code   | cust_name   | cust_city   | cust_country   |   grade |   outstanding_amt | agent_code   |
+=============+=============+=============+================+=========+===================+==============+
| C00001      | Micheal     | New York    | USA            |       2 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00020      | Albert      | New York    | USA            |       3 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00025      | Ravindran   | Bangalore   | India          |       2 |              8000 | A011         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00002      | Bolt        | New York    | US

#### We can continue querying the same view of the US and India customers. This time I want to check for any customers who have an outstanding balance over 3000 and residing in New York.

In [16]:
# Query to fetch records
cur.execute("""
    SELECT * 
    FROM customers_USA_india
    WHERE outstanding_amt > 3000 AND cust_city = 'New York'
""")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| cust_code   | cust_name   | cust_city   | cust_country   |   grade |   outstanding_amt | agent_code   |
+=============+=============+=============+================+=========+===================+==============+
| C00001      | Micheal     | New York    | USA            |       2 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00020      | Albert      | New York    | USA            |       3 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+


### 2.3 Generalization
#### Generalization can help create views that combine two similar categories into one. In this example, we want to query a table containing both agents and contractors. To do this, we can combine the two tables into one view called **all_employees**. 

In [17]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW all_employees AS 
        SELECT *
        FROM agents
        UNION
        SELECT * 
        FROM contractors
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [18]:
# Query to fetch records 
cur.execute("SELECT * FROM all_employees")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+--------------+--------------+----------------+--------------+--------------+-----------+
| agent_code   | agent_name   | working_area   |   commission | phone_no     | country   |
+==============+==============+================+==============+==============+===========+
| C003         | Liam         | Dublin         |         0.14 | 011-33447076 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A001         | Subbarao     | Bangalore      |         0.14 | 077-12346674 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A005         | Anderson     | Brisbane       |         0.13 | 045-21447739 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A012         | Lucida       | San Jose       |         0.12 | 044-52981425 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+